# AICE1006 - Data Analytics

## Lecture 5 - Data Wrangling


**Zhiwu Huang**  <br/>
Lecturer (Assistant Professor) <br/>
Vision, Learning and Control (VLC) Research Group <br/>
School of Electronics and Computer Science (ECS) <br/>
University of Southampton<br/>

*Office Hour: Wed 2PM-3PM, Please book in advance.* <br/>
``Zhiwu.Huang@soton.ac.uk``

<br/>
<br/>
<!-- <br/> -->

Credit: Marco Forgione, Researcher, USI-SUPSI


# Pandas Data Wrangling

### Pivoting, contatenating, and joining datasets

### Data Wrangling

In the data science/engineering jargon, **data wrangling** refers to the process of manipulating unstructured/messy data into a clean and structured form useful for the subsequent steps.

* Sometimes the original dataset is not in the ideal format for our analyses. We need to **apply transformations** to make it usable

* Sometimes the data we need comes from **multiple sources**. We need to **combine** them to a single dataframe

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
np.random.seed(42) # make (pseudo)-random numbers reproducible

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)

### Long and wide data format



Datasets naturally **indexed by a key** may be stored either in wide format or in long (aka narrow) format. <br/>Example: time series, where the key is the time instant.



In **wide format**, each time instant is a row. There is one column per variable.

In [4]:
df_wide = pd.DataFrame({"time": np.arange(3)*0.1, 'A': np.random.rand(3), 'B': np.random.rand(3), 'C': np.random.rand(3)})
df_wide

,time,A,B,C
0,0.0,0.374540,0.598658,0.058084
1,0.1,0.950714,0.156019,0.866176
2,0.2,0.731994,0.155995,0.601115


In **long format**, each row represents a single variable at a given time instant.

In [5]:
df_long = df_wide.melt("time")
df_long

,time,variable,value
0,0.0,A,0.374540
1,0.1,A,0.950714
2,0.2,A,0.731994
3,0.0,B,0.598658
4,0.1,B,0.156019
5,0.2,B,0.155995
6,0.0,C,0.058084
7,0.1,C,0.866176
8,0.2,C,0.601115


The wide format looks more intuitive. However, it is not always desirable:
 * if the time stamps of A, B, C are not the same, we need to include several NaNs in the table
 * if a new signal D is added, we need to add a new column to the dataframe

### Pivoting

To go from long to wide format, use the ``pivot`` dataframe method:

In [6]:
# df_wide_ = df_long.pivot("time", "variable", "value").reset_index()
# display('df_long', 'df_wide_')
df_wide_ = df_long.pivot(index="time", columns="variable", values="value").reset_index()
display('df_long', 'df_wide_')

df_long
   time variable     value
0   0.0        A  0.374540
1   0.1        A  0.950714
2   0.2        A  0.731994
3   0.0        B  0.598658
4   0.1        B  0.156019
5   0.2        B  0.155995
6   0.0        C  0.058084
7   0.1        C  0.866176
8   0.2        C  0.601115

df_wide_
variable  time         A         B         C
0          0.0  0.374540  0.598658  0.058084
1          0.1  0.950714  0.156019  0.866176
2          0.2  0.731994  0.155995  0.601115

The ``melt`` method does the inverse transformation:

In [7]:
df_wide_.melt("time")

,time,variable,value
0,0.0,A,0.374540
1,0.1,A,0.950714
2,0.2,A,0.731994
3,0.0,B,0.598658
4,0.1,B,0.156019
5,0.2,B,0.155995
6,0.0,C,0.058084
7,0.1,C,0.866176
8,0.2,C,0.601115


###  Long and wide dataset visualizations with plotly


With plotly, we can visualize datasets both in long and wide format

In [8]:
display('df_long', 'df_wide')

df_long
   time variable     value
0   0.0        A  0.374540
1   0.1        A  0.950714
2   0.2        A  0.731994
3   0.0        B  0.598658
4   0.1        B  0.156019
5   0.2        B  0.155995
6   0.0        C  0.058084
7   0.1        C  0.866176
8   0.2        C  0.601115

df_wide
   time         A         B         C
0   0.0  0.374540  0.598658  0.058084
1   0.1  0.950714  0.156019  0.866176
2   0.2  0.731994  0.155995  0.601115

In [9]:
fig = px.line(df_long, x="time", y="value", color="variable", width=800, height=300) # bar plot for a long dataset
fig.update_layout(font_size=18); fig.show()

In [10]:
fig = px.line(df_wide, x="time", y=["A", "B", "C"], width=800, height=300) # bar plot for a wide dataset
fig.update_layout(font_size=20); fig.show()

### Concatenating

It is possible to concatenate dataframes along rows or columns

In [11]:
df_data_0 = pd.DataFrame(np.random.rand(3, 2), columns=['foo', 'bar'])
df_data_1 = pd.DataFrame(np.random.rand(2, 2), columns=['foo', 'bar'])
display('df_data_0', 'df_data_1')

,foo,bar
0,0.708073,0.020584
1,0.969910,0.832443
2,0.212339,0.181825
,foo,bar
0,0.183405,0.304242
1,0.524756,0.431945


Let us **concatenate** the two dataframes along the rows. Syntax: ``pd.concat(df_list, axis=0)``.

In [12]:
df_data_concat = pd.concat([df_data_0, df_data_1]) # by default, concatenate along rows
#df_data_concat = pd.concat([df_data_0, df_data_1], axis=0) # by default, concatenate along rows

In [13]:
display('df_data_0', 'df_data_1', 'df_data_concat')

,foo,bar
0,0.708073,0.020584
1,0.969910,0.832443
2,0.212339,0.181825
,foo,bar
0,0.183405,0.304242
1,0.524756,0.431945
,foo,bar
0,0.708073,0.020584
1,0.969910,0.832443
2,0.212339,0.181825


Note: we have **duplicate index values** in the result! Not an issue for ``pandas``, but sometimes not desirable. 


### Concatenating


By setting the option ``ignore_index=True``, a new index for the concatenated dataframe is created

In [14]:
df_data = pd.concat([df_data_0, df_data_1], ignore_index=True)

display('df_data_0', 'df_data_1', 'df_data')

,foo,bar
0,0.708073,0.020584
1,0.969910,0.832443
2,0.212339,0.181825
,foo,bar
0,0.183405,0.304242
1,0.524756,0.431945
,foo,bar
0,0.708073,0.020584
1,0.969910,0.832443
2,0.212339,0.181825


### Concatenating


To concatenate along columns, specify the ``axis=1`` option:

In [15]:
df_data = pd.concat([df_data_0, df_data_1], axis=1) # default: axis=0
display('df_data_0', 'df_data_1', 'df_data')

df_data_0
        foo       bar
0  0.708073  0.020584
1  0.969910  0.832443
2  0.212339  0.181825

df_data_1
        foo       bar
0  0.183405  0.304242
1  0.524756  0.431945

df_data
        foo       bar       foo       bar
0  0.708073  0.020584  0.183405  0.304242
1  0.969910  0.832443  0.524756  0.431945
2  0.212339  0.181825       NaN       NaN

Note: 
 * Duplicate column names! Almost always not desirable...
 * NaN values are added at index 2

In practice, concatenation along columns is not very common...

### Append

the ``append`` dataframe method is an alternative method for row concatenation

In [16]:
# df_data = df_data_0.append(df_data_1) # = pd.concat(df_data_0, df_data_1)
# display('df_data_0', 'df_data_1', 'df_data')
df_data = pd.concat([df_data_0, df_data_1], ignore_index=True)
display('df_data_0', 'df_data_1', 'df_data')

,foo,bar
0,0.708073,0.020584
1,0.969910,0.832443
2,0.212339,0.181825
,foo,bar
0,0.183405,0.304242
1,0.524756,0.431945
,foo,bar
0,0.708073,0.020584
1,0.969910,0.832443
2,0.212339,0.181825


Note: unlike the Python list append, dataframe append returns a new object

In [17]:
char_lst = ['a', 'b', 'c']
char_lst.append('d') # list append modifies the current object. It is an inplace operation.
char_lst

['a', 'b', 'c', 'd']

In [18]:
# df_data = df_data_0.append(df_data_1) # dataframe append returns a new object and does not change the current object!
# df_data_0 # no change!
df_data = pd.concat([df_data_0, df_data_1], ignore_index=True)
df_data_0  # No change to df_data_0

,foo,bar
0,0.708073,0.020584
1,0.969910,0.832443
2,0.212339,0.181825


Always ask yourself whether the operation has *inplace* behavior. This can save you hours of debug time...

### Join operations

Join refers to powerful *relational algebra* operations available in pandas (and in most databases). 

* Two dataframes, named LEFT and RIGHT, are considered
* A *join* expression involving one or more *keys* is evaluated for all combination of rows in the left and right dataframes
* The resulting dataframe contains all the rows from LEFT/RIGHT where the join expression is satisfied

A formal description of all join operations is beyond the scope of the course. We make some examples...

<center>
<p align="center">
 <img src="img/generic_join.png" alt="generic_join" width=1200> 
</p>
</center>

* The key variable is **Customer**
* The join expression above is **equality**: LEFT.Customer == RIGHT.Customer. This is an **equi-join** (as all the following examples).
* It works even though customers are in different order in the two tables. Column-wise concatenation would produce a wrong result!

### Inner join

Join comes in different flavors. In an **inner join**, we only consider keys existing both in the LEFT *and* the RIGHT dataframe.

<center>
<p align="center">
 <img src="img/inner_join.png" alt="inner_join" width=1200> 
</p>
</center>

Inner join makes sense when we do not want to have missing values in the result.

### Left join

The rows in the LEFT dataframe are preserved in the result, even when they do not exist in the RIGHT dataframe.

<center>
<p align="center">
 <img src="img/left_join.png" alt="left_join" width=1200> 
</p>
</center>

Left join makes sense if we do not want to lose information from the LEFT dataframe.

### Right  join

The rows in the RIGHT dataframe are preserved in the result, even when they do not exist in the LEFT dataframe.

<center>
<p align="center">
 <img src="img/right_join.png" alt="right_join" width=1200> 
</p>
</center>

Right join makes sense if we do not want to lose information from the RIGHT dataframe.

### Outer join

The rows of both dataframes are preserved in the result.

<center>
<p align="center">
 <img src="img/outer_join.png" alt="outer_join" width=1400> 
</p>
</center>

Outer join makes sense if we do not want to lose information from the two dataframes.

### Join "One-to-many"

Previous join examples were all *one-to-one*. At most one key from LEFT matched with one key from RIGHT. 

Here is a one-to-many example:

<center>
<p align="center">
 <img src="img/one_to_many_join.png" alt="one_to_many_join" width=1400> 
</p>
</center>

One-to-many join operations are common in practice. The more general case *many-to-many* is also possible...

### Join operations in pandas

The ``merge`` dataframe method is the main tool to perform join operations in pandas

In [19]:
df_left = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Ann'], 'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'Engineering']})
df_right = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'], 'hire_date': [2004, 2008, 2012, 2014]})
display('df_left', 'df_right')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Ann,Engineering
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [20]:
df_result = df_left.merge(df_right) # df_result = df_left.merge(df_right, on="employee", how="inner") # equivalent in this case

In [21]:
display('df_left', 'df_right', 'df_result')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Ann,Engineering
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


By default:
* The keys are the columns which are present in left and right dataframe: **employee** in this example
* The join expression is equality: rows corresponding to the same **employee** from left/right are included in the result
* The result only includes keys that are present in both dataframes (inner join). Employee Ann is excluded from the result.

In [22]:
# Equivalent (more explicit) expression
df_result = df_left.merge(df_right, on="employee", how="inner") # equivalent in this case

### Join operations in pandas

Column names are often not consistent in real data. 

Use the ``left_on`` and ``right_on`` options to specify the left and right keys to be compared

In [23]:
df_left = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Ann'], 'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'Engineering']})
df_right = pd.DataFrame({'Employee': ['Lisa', 'Bob', 'Jake', 'Sue'], 'hire_date': [2004, 2008, 2012, 2014]})

In [24]:
df_join = df_left.merge(df_right, left_on="employee", right_on="Employee")

In [25]:
display('df_left', 'df_right', 'df_join')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Ann,Engineering
,Employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


We can now drop the redundant column **Employee** and get the desired result:

In [26]:
df_join.drop(columns="Employee") # remove either employee or Employee

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### Join operations in pandas

We may want to perform left/right/inner/outer join. Use the ``how`` option accordingly:

In [27]:
df_left = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Ann'], 'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'Engineering']})
df_right = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue', 'Marco'], 'hire_date': [2004, 2008, 2012, 2014, 2018]})

In [28]:
df_inner_join = df_left.merge(df_right, how="inner") # default: inner join
df_left_join = df_left.merge(df_right, how="left") # left outer join
df_right_join = df_left.merge(df_right, how="right") # right outer join
df_outer_join = df_left.merge(df_right, how="outer") # full outer join

display('df_left', 'df_right', 'df_inner_join', 'df_left_join', 'df_right_join', 'df_outer_join')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Ann,Engineering
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


* ``inner``: keep only keys in both dataframes (default)
* ``left``:  keep all keys in left dataframe
* ``right``: keep all keys in right dataframe
* ``outer``: keep all keys

### Join on index

All join operations seen so far are performed using the ``merge`` dataframe method. 

Another method called ``join`` actually exist. It refers to a join operation applied to the dataframes *index*.

In [29]:
df_left = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Ann'], 'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'Engineering']}).set_index("employee")
df_right = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue', 'Marco'], 'hire_date': [2004, 2008, 2012, 2014, 2018]}).set_index("employee")

In [30]:
display('df_left', 'df_right')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
Ann,Engineering
,hire_date
employee,
Lisa,2004
Bob,2008


In [31]:
df_left.join(df_right, how="inner") # inner join on the dataframe index

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014
